In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import risk_kit as rk
from pricing_models import StockPricing, FuturesPricing, OptionsPricing

Questions 1-8 should be answered by building a 15-period binomial model whose parameters should be calibrated to a Black-Scholes geometric Brownian motion model with: T = .25 years, $S_{0}$ = 100, r = 2%, σ = 30% and a dividend yield of c = 1%.

In [2]:
# Setting parameters
periods = 15
T = 0.25
S0 = 100.0
r = 0.02
sigma = 0.3
c = 0.01

In [3]:
binomial_params = rk.blackscholes_to_binomial(r, sigma, periods, T, c)
u = binomial_params['upward_drift']
d = binomial_params['downward_drift']
div = binomial_params['dividend_per_period']
rfr = binomial_params['risk_free_rate']

pricing_model = StockPricing(n=periods,S0=S0,u=u,d=d,c=div)

### Question 1

Compute the price of an American call option with strike K=110 and maturity T=.25 years.

In [4]:
K = 110
is_call = True
is_american = True

q = ((1+rfr) - d - div) / (u - d)

call_option = OptionsPricing(n=periods,
                           model=pricing_model,
                           r=rfr,
                           q=q,
                           K=K,
                           is_call=is_call,
                           is_american=is_american)
round(call_option.price, 2)

2.6

### Question 2

Compute the price of an American put option with strike K=110 and maturity T=.25 years.

In [5]:
K = 110
is_call = False
is_american = True

q = ((1+rfr) - d - div) / (u - d)

put_option = OptionsPricing(n=periods,
                           model=pricing_model,
                           r=rfr,
                           q=q,
                           K=K,
                           is_call=is_call,
                           is_american=is_american)
round(put_option.price, 2)

12.36

### Question 3

Is it ever optimal to early exercise the put option of Question 2?

In [6]:
answer = True if put_option.early_exercise else False
answer

True

### Question 4

If your answer to Question 3 is "Yes", when is the earliest period at which it might be optimal to early exercise?

In [7]:
put_option.early_exercise[0]['Time']

5

### Question 5

Do the call and put option prices of Questions 1 and 2 satisfy put-call parity?

$No$, the parity is valid only for european options

### Question 6

Compute the fair value of an American call option with strike K = 110 and maturity n=10 periods where the option is written on a futures contract that expires after 15 periods. 

The futures contract is on the same underlying security of the previous questions.

In [8]:
futures_pricing = FuturesPricing(n=periods,
                                model=pricing_model,
                                q=q)

In [9]:
options_periods = 10
is_call = True
K = 110
is_american = True

option_on_futures = OptionsPricing(n=options_periods,
                                  model=futures_pricing,
                                  r=rfr,
                                  q=q,
                                  K=K,
                                  is_call=is_call,
                                  is_american=is_american)
round(option_on_futures.price, 2)

1.66

### Question 7

What is the earliest time period in which you might want to exercise the American futures option of Question 6?

In [10]:
option_on_futures.early_exercise[0]['Time']

7

### Question 8

Compute the fair value of a $chooser$ option which expires after n = 10 periods. At expiration the owner of the chooser gets to choose (at no cost) a European call option or a European put option. The call and put each have strike K = 100 and they mature 5 periods later, i.e. at n=15.

In [11]:
periods = 15
K = 100
call_option = OptionsPricing(n=periods,
                            model=pricing_model,
                            r=rfr,
                            q=q,
                            K=K,
                            is_call=True)

put_option = OptionsPricing(n=periods,
                            model=pricing_model,
                            r=rfr,
                            q=q,
                            K=K,
                            is_call=False)

chooser_period = 10
tree = np.zeros([chooser_period + 1, chooser_period + 1])

for i in range(chooser_period, -1, -1):
    if i==chooser_period:
        for j in range(i + 1):
            tree[i, j] = max(call_option.tree[i, j], put_option.tree[i, j])
    else:
        for j in range(i + 1):
            childd = tree[i + 1, j]
            childu = tree[i + 1, j + 1]
            
            price = (q * childu + (1 - q) * childd) / (1 + rfr)
            
            tree[i, j] = price
round(tree[0, 0], 2)

10.81